In [78]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import matplotlib as mlp
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from matplotlib.patches import Rectangle, Ellipse
from rich import print
from time import time

from typing import List, Dict, Union

from read_csv import read_meta_info

sns.set_theme('notebook')
sns.set_style("whitegrid")
sns.set_context("paper")
sns.color_palette("hls", 8)

def print_bl():
    print("\n")

In [125]:
class PromptExtractor:
    """
    A class that represents an entire recording of the highD dataset.

    Attributes:
        dataset_location: a path to the directory in which the dataset is stored.
        dataset_index: index of the recording that is to be addressed
        sampling_period: time period between sampled frames. Needs to be a multiple of 40 ms    
    
    """
    def __init__(self, dataset_location: str = None, dataset_index: int = None):
        self.dataset_index  = dataset_index
        self.dataset_location = dataset_location
        self.frame_length = 40 # each frame is 40 ms
        self.df_location = dataset_location + str(dataset_index).zfill(2) + "_tracks.csv"
        self.static_info_location = dataset_location + str(dataset_index).zfill(2) + "_tracksMeta.csv"
        self.video_info_location = dataset_location + str(dataset_index).zfill(2) + "_recordingMeta.csv"

        self.data = pd.read_csv(self.df_location)
        self.static_info = pd.read_csv(self.static_info_location)
        self.video_info = read_meta_info(self.video_info_location)

    def filter_data(self, sampling_period: int = 1000) -> pd.DataFrame:
        """
        Filters the dataset with a certain sampling period.

        Args:
            sampling_period (int): The time spacing between each frame to consider.

        Returns:
            A list of dictionaries with one frame per dictionary.
        """
        
        self.sampling_period = sampling_period
        self.frame_spacing = int(self.sampling_period / self.frame_length) #Frames are 40 ms apart

        # Filter data
        self.data = self.data[self.data.frame % self.frame_spacing == 0]
        self.data.frame = self.data.frame / self.frame_spacing
        self.data = self.data.astype({'frame': 'int16'})

    def get_frame_windows(self, window_size: int = 5) -> List[pd.DataFrame]:
        """
        Splits the dataset into windows of a certain size.

        Args:
            window_size (int): The size of the window to consider.

        Returns:
            A list of dataframes with the windows.
        """

        self.window_size = window_size
        self.windows = []

        for i in range(1, len(self.data), window_size):
            window = self.data[self.data['frame'].isin(range(i, i+window_size))]
            self.windows.append(window) #no overlap
        self.windows[0].head()

    def get_groups(self, bubble_radius: float = 50.0, lookback: int = 5):
        def in_bubble(ego_vehicle, x, radius = bubble_radius):
            '''
            Calculates whether a vehicle is in the bubble of the ego vehicle. Checks distance between ego vehicle and target, as well as driving direction.
            Notation: 
                (x, y) - coordinates of the top left corner of the bounding box of the vehicle.
                (w, h) - width and height of the bounding box.

                Parameters:
                    ego_vehicle: dataframe row with information about the ego vehicle
                    x: dataframe row with information about the vehicle to consider
                Returns:
                    bool: True if the vehicle is in the bubble, False otherwise
                    
            '''
            x1, y1, w1, h1 = ego_vehicle.x, ego_vehicle.y, ego_vehicle.width, ego_vehicle.height
            x2, y2, w2, h2 = x.x, x.y, x.width, x.height
            c1 = np.array([x1 + w1/2, y1 + h1/2])
            c2 = np.array([x2 + w2/2, y2 + h2/2])
            dist = np.linalg.norm(c1-c2)

            return dist < radius & np.sign(x.xVelocity) == np.sign(ego_vehicle.xVelocity)
        
        self.bubble_radius = bubble_radius
        self.lookback = lookback
        
        self.groups = []

        for window in self.windows:
            #choose ego vehicle - one present in every frame
            ego_candidates = []
            id_counts = window.id.value_counts().to_dict()
            print(id_counts)

            break
            

In [126]:
dataset_location = "/home/lmmartinez/Tesis/datasets/highD/data/"
dataset_index = 1
start = time()
scene_data = PromptExtractor(dataset_location=dataset_location, dataset_index=dataset_index)
end = time()
print("Time elapsed is:", end - start)
start = time()
scene_data.filter_data(sampling_period=1000)
scene_data.get_frame_windows(window_size=5)
scene_data.get_groups()
end = time()
print("Time elapsed is:", end - start)

Time elapsed is: 0.16544485092163086

{9: 5, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 5, 2: 5, 10: 5, 11: 5, 12: 5, 15: 4, 14: 3, 16: 2, 17: 2, 13: 1, 1: 1}

Time elapsed is: 0.8104608058929443

In [117]:
scene_data.windows[0].to_csv("window.csv")